# Cascade simulator demo (range 1–23)

Этот ноутбук показывает базовый сценарий работы симулятора каскадов:

1. Генерация синтетического сигнала (сумма синусоид).
2. Расчёт спектра (freq, power).
3. Загрузка публичной шкалы каскадов 1–23.
4. Проекция спектра в каскадное пространство и выбор доминирующего каскада.
5. Визуализация временного сигнала, спектра и вектора каскадов.

In [ ]:
%matplotlib inline

import sys
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

# Добавляем корень проекта в sys.path, чтобы модуль src был доступен из notebooks/
PROJECT_ROOT = Path("..").resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

from src.core.signal_loader import generate_test_signal, compute_spectrum
from src.core.cascade_scale import get_public_scale
from src.core.cascade_mapping import map_dominant_cascade


## 1. Генерация тестового сигнала

Сделаем простую модель: сумма двух синусоид (2 Гц и 5 Гц) с небольшим шумом.

In [ ]:
t, x = generate_test_signal(
    duration=10.0,
    dt=0.01,
    freqs=(2.0, 5.0),
    amplitudes=(1.0, 0.5),
    seed=42,
)

print(f"Signal length: {len(t)} samples")

# Визуализируем небольшой фрагмент временного сигнала
plt.figure(figsize=(8, 3))
n_show = 500
plt.plot(t[:n_show], x[:n_show])
plt.xlabel("Time, s")
plt.ylabel("Amplitude")
plt.title("Time-domain signal (fragment)")
plt.tight_layout()
plt.show()


## 2. Спектр сигнала

Считаем спектр с помощью БПФ и смотрим, где сосредоточена мощность.

In [ ]:
freq, power = compute_spectrum(t, x)

plt.figure(figsize=(8, 3))
plt.plot(freq, power)
plt.xlim(0, 10)
plt.xlabel("Frequency, Hz")
plt.ylabel("Power")
plt.title("Power spectrum (0–10 Hz)")
plt.tight_layout()
plt.show()


## 3. Проекция в каскадную шкалу 1–23

Теперь загрузим публичную шкалу каскадов, применим `map_dominant_cascade` и посмотрим:

- номер доминирующего каскада,
- уверенность (доля мощности),
- распределение по всем 23 уровням.

In [ ]:
scale = get_public_scale()
result = map_dominant_cascade(freq, power, scale)

main_cascade = result["main_cascade"]
confidence = result["confidence"]
vector = result["vector"]

print("Dominant cascade:", main_cascade)
print("Confidence (fraction of total power):", round(confidence, 3))

cascade_ids = [lvl.cascade_id for lvl in scale]

plt.figure(figsize=(10, 4))
plt.bar(cascade_ids, vector)
plt.xlabel("Cascade level (1–23)")
plt.ylabel("Weight")
plt.title("Cascade weight vector for test signal")
plt.tight_layout()
plt.show()


Далее можно экспериментировать:

- менять частоты и амплитуды синусоид,
- использовать реальные временные ряды вместо `generate_test_signal`,
- сравнивать разные сигналы по их положению в каскадной шкале 1–23.